In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
## Load the trained model, scaler, pickle, onehot
model = load_model('model.h5')

## Load the encoder and scaler
with open('onehotencoder.pkl', 'rb') as file:
    onehotencoder = pickle.load(file)

with open('scalar.pkl', 'rb') as file:
    scalar = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)


In [5]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [6]:
## One-Hot Encode Geography
geo_encoded = onehotencoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehotencoder.get_feature_names_out(['Geography']))
geo_encoded_df


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
## Encode the categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
## Concatenation with onehot encoded data
if "Geography" in input_df.columns:
    input_df = input_df.drop("Geography", axis=1)
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
##Scaling the input data
input_scaled = scalar.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [13]:
## Predict Churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step


array([[0.04222566]], dtype=float32)

In [14]:
prediction_probab = prediction[0][0]

In [15]:
prediction_probab

np.float32(0.04222566)

In [16]:
if prediction_probab > 0.5:
  print("The customer is likely to churn.")
else:
  print("The customer is not likely to churn.")

The customer is not likely to churn.
